## openpose生图

此文件的作用是使用openpose的controlnet，以源文件夹中的图片为参考，加上指定的prompt，生成新的图片。

In [1]:
import json
from urllib import request, parse
import random
import uuid
import urllib.request
import urllib.parse
import io
import os
import time
from PIL.PngImagePlugin import PngInfo
import hashlib
from urllib.error import URLError


## import json file

In [2]:
promptJson = 'api.json'

with open(promptJson) as f:
    prompt = json.load(f)

print(str(prompt))

{'4': {'inputs': {'ckpt_name': 'fudukiMix_v15.safetensors'}, 'class_type': 'CheckpointLoaderSimple'}, '8': {'inputs': {'samples': ['93', 0], 'vae': ['60', 2]}, 'class_type': 'VAEDecode'}, '39': {'inputs': {'model': ['4', 0], 'clip': ['4', 1], 'lora_stack': ['40', 0]}, 'class_type': 'CR Apply LoRA Stack'}, '40': {'inputs': {'switch_1': 'On', 'lora_name_1': 'miho_v1.safetensors', 'model_weight_1': 1, 'clip_weight_1': 1, 'switch_2': 'On', 'lora_name_2': 'NsfwPovAllInOneLoraSdxl-000009.safetensors', 'model_weight_2': 1, 'clip_weight_2': 0.4, 'switch_3': 'Off', 'lora_name_3': 'None', 'model_weight_3': 1, 'clip_weight_3': 0.4}, 'class_type': 'CR LoRA Stack'}, '44': {'inputs': {'image': '20230520004 (8).jpg', 'choose file to upload': 'image'}, 'class_type': 'LoadImage'}, '46': {'inputs': {'images': ['70', 0]}, 'class_type': 'PreviewImage'}, '48': {'inputs': {'text': 'analog film grain,BREAK (TOK:1.5),a photo of japanese girl,she cute baby face,flat chest,shaved pussy,and big ass.', 'clip': ['

In [3]:
from enum import Enum, unique

@unique
class RunMode(Enum):
    LOCAL = 1
    POD_LOCAL = 2
    POD_REMOTE = 3

run_mode = RunMode.LOCAL
# 设置运行模式，来实现在本机，runpod本地，runpod远程的切换
server_address = "2bzezpggl7sjvi-3000.proxy.runpod.net/" if run_mode == RunMode.POD_REMOTE else "0.0.0.0:3000" if run_mode == RunMode.POD_LOCAL else "127.0.0.1:8188"
client_id = str(uuid.uuid4())


In [4]:
def changeImageSize(width, height):
    # 最小的宽高为本机896，runpod1024
    limited = 896 if run_mode == RunMode.LOCAL else 1024
    minSize = min(width, height)

    # 缩放宽高，使得最窄的边等于limited
    if minSize > limited:
        scale = minSize / limited
        widthScale = int(width / scale)
        heightScale = int(height / scale)
    else:
        scale = limited / minSize
        widthScale = int(width * scale)
        heightScale = int(height * scale)

    # 将两个值都除以64，然后取整,也就是说边长必须是64的倍数
    widthScale = int(widthScale/64) * 64
    heightScale = int(heightScale/64) * 64

    return widthScale, heightScale

def checkImageExsit(imageDir, imageName):
    for file in os.listdir(imageDir):
        if file.startswith(imageName):
            return True
    return False


def queue_prompt(prompt):
    while True:
        try:
            p = {"prompt": prompt, "client_id": client_id}
            data = json.dumps(p).encode('utf-8')
            req = urllib.request.Request("http://{}/prompt".format(server_address), data=data)
            return json.loads(urllib.request.urlopen(req, timeout=5).read())
        except URLError:
            print("连接超时，正在重试...")

def get_history(prompt_id):
    with urllib.request.urlopen("http://{}/history/{}".format(server_address, prompt_id)) as response:
        return json.loads(response.read())

def get_image(filename, subfolder, folder_type):
    data = {"filename": filename, "subfolder": subfolder, "type": folder_type}
    url_values = urllib.parse.urlencode(data)
    with urllib.request.urlopen("http://{}/view?{}".format(server_address, url_values)) as response:
        return response.read()

def handle_whitespace(string: str):
    return string.strip().replace("\n", " ").replace("\r", " ").replace("\t", " ")

def parse_name(ckpt_name):
    path = ckpt_name
    filename = path.split("/")[-1]
    filename = filename.split(".")[:-1]
    filename = ".".join(filename)
    return filename


def calculate_sha256(file_path):
    sha256_hash = hashlib.sha256()

    with open(file_path, "rb") as f:
        # Read the file in chunks to avoid loading the entire file into memory
        for byte_block in iter(lambda: f.read(4096), b""):
            sha256_hash.update(byte_block)

    return sha256_hash.hexdigest()

def get_images(ws, prompt):
    prompt_id = queue_prompt(prompt)['prompt_id']
    output_images = {}
    while True:
        out = ws.recv()
        if isinstance(out, str):
            message = json.loads(out)
            if message['type'] == 'executing':
                data = message['data']
                if data['node'] is None and data['prompt_id'] == prompt_id:
                    break #Execution is done
        else:
            continue #previews are binary data

    history = get_history(prompt_id)[prompt_id]
    for o in history['outputs']:
        for node_id in history['outputs']:
            node_output = history['outputs'][node_id]
            if 'images' in node_output:
                images_output = []
                for image in node_output['images']:
                    image_data = get_image(image['filename'], image['subfolder'], image['type'])
                    images_output.append(image_data)
                output_images[node_id] = images_output

    return output_images
    
def saveImages(result, imageDir,imageName,comment,previewImage=0): 
    metadata = PngInfo()
    metadata.add_text("parameters", comment)  
    k = 0 
    for node_id in result:    
        if k < previewImage:
            k += 1
            continue    
        for image_data in result[node_id]:            
            image = Image.open(io.BytesIO(image_data))
            imagePath = imageDir + "/" + imageName + f"_{k}" + ".png"
            image.save(imagePath,pnginfo=metadata) 
            k += 1
    

In [5]:
# 子文件夹
subDir = "BoA"

# 原图文件夹
originalDir = f"f:/Poledriver/Paid/{subDir}" if run_mode != RunMode.POD_LOCAL else f"../../../image-inputs/{subDir}"
# 设置图片保存路径
imageDir = f"f:/Poledriver/remake/{subDir}" if run_mode != RunMode.POD_LOCAL else f"../../../image-outputs/{subDir}"
# 如果文件夹不存在，创建文件夹
if not os.path.exists(imageDir):
    os.mkdir(imageDir)

In [6]:

from PIL import Image
%pip install sd-parsers
from sdparsers import ParserManager
import websocket

ws = websocket.WebSocket()
ws.connect("ws://{}/ws?clientId={}".format(server_address, client_id))

batch_size = 4
# 遍历原图文件夹里所有的图片
for originalImag in os.listdir(originalDir):
        # 如果不是jpg或png文件，跳过
        if not (originalImag.endswith(".jpg") or originalImag.endswith(".png") or originalImag.endswith(".jpeg")):
            continue
        
        originalImagePath = originalDir + "/" + originalImag
        # 如果文件名包含_H或者_B，跳过
        if originalImag.find("_H") != -1 or originalImag.find("_B") != -1:
            print(f"{originalImag}包含_H或者_B,判定为mask文件，跳过")
            continue
        
        # 读取原图片名字，去掉后缀.png
        originalImagName = originalImag.split(".")[0]
        # 检测目标文件夹里是否已经存在该图片，如果存在，跳过
        if checkImageExsit(imageDir, originalImagName):
            print(f"{originalImagName}已经存在，跳过")
            continue  
        
        # 读取原图，以及其大小
        with Image.open(originalImagePath) as originalImage:
            originalImageWidth = originalImage.size[0]
            originalImageHeight = originalImage.size[1]
            parser_manager = ParserManager()
            prompt_data = parser_manager.parse(originalImage)
            # 读取原图的提示词，如果解析失败，使用默认提示词
            try:
                promt_string = prompt_data.prompts[0][0].value  
                prompt['6']['inputs']['text'] = promt_string
                negative_string = prompt_data.prompts[0][1].value
                prompt['7']['inputs']['text'] = negative_string
            except:
                print(f"原图片{originalImag}解析信息失败,使用默认提示词")
              
        
        width,height = changeImageSize(originalImageWidth, originalImageHeight)  
        print(f"原图名字：{originalImagName}。大小：{originalImageWidth}x{originalImageHeight}")
        print(f"缩放后的图片大小是{width}*{height}")
        megapixels = width / originalImageWidth
        seed = random.randint(0, 976242998978323)
        
        # 下面是确定实际生成图片的大小，种子，以及保存图片信息
        prompt['44']['inputs']['image'] = originalImagePath
        prompt['93']['inputs']['noise_seed']= seed        
        prompt['95']['inputs']['megapixels'] = megapixels
        prompt['96']['inputs']['amount'] = batch_size

        steps = prompt['93']['inputs']['steps']
        cfg = prompt['93']['inputs']['cfg']
        sampler_name = prompt['93']['inputs']['sampler_name']
        scheduler = prompt['93']['inputs']['scheduler']
        positive = prompt['48']['inputs']['text']
        negative = prompt['49']['inputs']['text']
        
        modelname = prompt['4']['inputs']['ckpt_name']
        basemodelname = parse_name(modelname)        
        ckpt_path = "e:/sdwebui/webui/models/Stable-diffusion/" + modelname if run_mode != RunMode.POD_LOCAL else "../../../ComfyUI/models/checkpoints/" + modelname
        modelhash = calculate_sha256(ckpt_path)[:10]
        
        comment = f"{handle_whitespace(positive)}\nNegative prompt: {handle_whitespace(negative)}\nSteps: {steps}, Sampler: {sampler_name}{f'_{scheduler}' if scheduler != 'normal' else ''}, CFG Scale: {cfg}, Seed: {seed}, Size: {width}x{height}, Model hash: {modelhash}, Model: {basemodelname}, Version: ComfyUI"    
        
        print(f"图片的信息是：{comment}")  
        
        # 连接api，生成并保存图片
        start_time = time.time()        
        images = get_images(ws, prompt)   
        saveImages(images, imageDir, originalImagName, comment=comment,previewImage=0)     
        end_time = time.time()
        print(f"耗时{end_time-start_time}秒")
        

20230414008 (9)已经存在，跳过
20230520001 (12)已经存在，跳过
20230520001 (16)已经存在，跳过
20230520001 (18)已经存在，跳过
20230520001 (2)已经存在，跳过
20230520001 (21)已经存在，跳过
20230520001 (3)已经存在，跳过
20230520001 (5)已经存在，跳过
20230520001 (6)已经存在，跳过
原图片20230520001 (7).jpg解析信息失败,使用默认提示词
原图名字：20230520001 (7)。大小：1920x1080
缩放后的图片大小是1536*896
